In [14]:
import random
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.dropout(torch.relu(self.fc_1(x)))
        x = self.fc_2(x)        
        return x

In [16]:
class SelfAtt(nn.Module):
  def __init__(self, emb_size, heads,dropout):
    super(SelfAtt,self).__init__()
    self.emb_size = emb_size
    self.heads = heads
    self.head_dim = emb_size//heads
    assert(self.head_dim*heads == self.emb_size), "head_dim*heads != emb_size"

    self.query = nn.Linear(self.emb_size,self.emb_size)
    self.key = nn.Linear(self.emb_size,self.emb_size)
    self.value = nn.Linear(self.emb_size,self.emb_size)

    self.fc_out = nn.Linear(self.head_dim*heads, self.emb_size)
    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
    self.dropout = nn.Dropout(dropout)

  def forward(self,values,keys,query,mask=None):
    N = query.shape[0]
    values = self.value(values)
    keys = self.key(keys)  
    query1 = self.query(query) 

    values = values.view(N, -1, self.heads, self.head_dim).permute(0, 2, 1, 3)
    keys = keys.view(N, -1, self.heads, self.head_dim).permute(0, 2, 1, 3)
    query1 = query1.view(N, -1, self.heads, self.head_dim).permute(0, 2, 1, 3)
    energy = torch.matmul(query1, keys.permute(0, 1, 3, 2)) / self.scale

    if mask is not None:
      energy = energy.masked_fill(mask == 0, float("-1e20"))
    
    attention = torch.softmax(energy, dim= -1)
    print(attention)
    x = torch.matmul(self.dropout(attention), values)    
    x = x.permute(0, 2, 1, 3).contiguous()
    out = x.view(N, -1, self.emb_size)
    
    out =  self.fc_out(out)
    return out

In [17]:
class TransformerBlock(nn.Module):
  def __init__(self,emb_size, heads, dropout, forward_expansion):
    super(TransformerBlock,self).__init__()
    self.att = SelfAtt(emb_size,heads,dropout)
    self.norm1 = nn.LayerNorm(emb_size)
    self.norm2 = nn.LayerNorm(emb_size)

    self.feed_forward = PositionwiseFeedforwardLayer(emb_size, forward_expansion*emb_size, dropout)
    self.dropout = nn.Dropout(dropout)

  def forward(self, value, key, query):
    att = self.att(value, key, query)
    x = self.norm1(query + self.dropout(att))
    forward = self.feed_forward(x)
    out  = self.norm2(x + self.dropout(forward))
    return out

In [18]:
class Encoder(nn.Module):
  def __init__(self, 
               src_vocab_size, 
               emb_size,
               num_layers,
               heads,
               device,
               forward_expansion,
               dropout,
               max_length,
               ):
    super(Encoder,self).__init__()
    self.emb_size = emb_size
    self.device = device
    self.word_embedding  = nn.Embedding(src_vocab_size,emb_size)
    self.position_embedding = nn.Embedding(max_length, emb_size)
    self.layers = nn.ModuleList(
        [
         TransformerBlock(
             emb_size,
             heads,
             dropout = dropout,
             forward_expansion = forward_expansion,
         )
         for _ in range(num_layers)
        ]
    )
    self.dropout = nn.Dropout(dropout)
    self.scale = torch.sqrt(torch.FloatTensor([emb_size])).to(device)
  def forward(self,x):
    N,seq_length = x.shape
    positions = torch.arange(0, seq_length).unsqueeze(0).repeat(N, 1).to(self.device)
    out = self.dropout(self.word_embedding(x)*self.scale + self.position_embedding(positions))
    for layer in self.layers:
      out  = layer(out,out,out)
      
    return out

In [19]:
class DecoderBlock(nn.Module):
  def __init__(self, emb_size, heads, forward_expansion, dropout, device):
    super(DecoderBlock,self).__init__()
    self.attention = SelfAtt(emb_size,heads,dropout)
    self.norm1 = nn.LayerNorm(emb_size)
    self.norm = nn.LayerNorm(emb_size)
    self.transformer_block = TransformerBlock(emb_size, heads, dropout, forward_expansion)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, value, key, trg_mask):
    attention = self.attention(x,x,x,trg_mask)
    query = self.norm1(self.dropout(attention) + x)
    out  = self.transformer_block(value, key, query)
    return out

In [20]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([embed_size])).to(device)

    def forward(self, x, enc_out, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).unsqueeze(0).repeat(N, 1).to(self.device)
        x = self.dropout((self.word_embedding(x)*self.scale) + self.position_embedding(positions))
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, trg_mask)

        out = self.fc_out(x)

        return out

In [21]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        embed_size=5,
        num_layers=6,
        forward_expansion=4,
        heads=1,
        dropout=0,
        device="cpu",
        max_length=10,
        src_pad_idx = 0,
        trg_pad_idx = 0,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.device = device
        self.trg_pad_idx = trg_pad_idx

    def make_trg_mask(self, trg):
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        trg_len = trg.shape[1]
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        trg_mask = trg_pad_mask & trg_sub_mask
        return trg_mask

    def forward(self, src, trg):
        trg_mask = self.make_trg_mask(trg)
        print("###########################trg_mask############################")
        print("")
        print("")
        print("")
        print(trg_mask)
        print("###########################Encoder############################")
        print("")
        print("")
        print("")
        enc_src = self.encoder(src)
        print("###########################Decoder############################")
        print("")
        print("")
        print("")
        out = self.decoder(trg, enc_src, trg_mask)
        return out


In [22]:
if __name__ == "__main__":
    print(device)
    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0]]).to(
        device
    )
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0]]).to(device)

    src_pad_idx = 1
    trg_pad_idx = 3
    src_vocab_size = 10
    trg_vocab_size = 10 
    model  = Transformer( src_vocab_size, trg_vocab_size,).to(device)
    out = model(x, trg)
    print(out.shape)

cpu
###########################trg_mask############################



tensor([[[[ True, False, False, False, False, False, False, False],
          [ True,  True, False, False, False, False, False, False],
          [ True,  True,  True, False, False, False, False, False],
          [ True,  True,  True,  True, False, False, False, False],
          [ True,  True,  True,  True,  True, False, False, False],
          [ True,  True,  True,  True,  True,  True, False, False],
          [ True,  True,  True,  True,  True,  True,  True, False],
          [ True,  True,  True,  True,  True,  True,  True, False]]]])
###########################Encoder############################



tensor([[[[0.1266, 0.1157, 0.0879, 0.0687, 0.1109, 0.0953, 0.1135, 0.1483,
           0.1330],
          [0.0815, 0.0943, 0.1822, 0.1760, 0.2046, 0.0615, 0.1153, 0.0267,
           0.0578],
          [0.1787, 0.0407, 0.0479, 0.1591, 0.2393, 0.0331, 0.0994, 0.0121,
           0.1896],
          [0.0191, 0.1495, 0.22